# 機械学習入門4：ハイパーパラメータチューニング(Hyperparameter Tuning)

In [ ]:
# パラメータ - 学習時実行後にモデルが獲得する値(重み)

# ハイパーパラメータ - 学習実行前に設定すべき,アルゴリズムの挙動を制御するための値
# 学習前にハイパーパラメータを調整することでモデルの性能向上,過学習の抑制,効率の良い学習が期待できる。

# ホールドアウト方 - 学習用データセットとテストデータセットの2分割

# しかし実際のモデルの開発時にはモデルの性能評価をより適切にするために、データを3分割してモデルを評価することが一般的
# 1,学習用データセット(train) - モデルを学習させるためのデータセット
# 2,検証用データセット(validation) - ハイパーパラメータの調整が適切なのか検証するためのデータセット
# 3,テスト用データセット(test) - 学習済みモデルの性能を評価するためのデータセット

# 1,2は学習段階で用いられる
# 3,は最終的なモデルの予測精度の確認のためにのみ使用する
# しかし十分なデータ量が用意できないと,３分割すると偏りが生まれて適切な学習,検証が行われない可能性がある
# そのようなデータの偏りを回避する方法としてk-分割交差検証(K-fold cross-validation)がある

# k-分割交差検証(K-fold cross-validation) - 3ステップ
# 1,データセットをk個に分割 (例 K=5)

# 2,分割したデータの1個の検証用データセットとし,残りk-1個の学習用データセットとして学習を実行 
# (例 K=5なので 4は学習用データセット,1はテスト用データセットになる k=5分の学習が分割して行われる(5回学習))
# そうすればデータに偏りなくハイパーパラメータのチューニングが可能

# 3,各検証の結果を平均して最終的な検証結果とする

# 前提 - k-分割交差検証は学習用データセットと検証用データセットの分割に用いることが多い


#ハイパーパラメータの調整方法

In [ ]:
# 1,手動での調整 - 手動での調整方法
# 2,グリッドサーチ - 
# 3,ランダムサーチ - ランダムにハイパーパラメータを調整していく方法
# 4,ベイズ最適化 - 他の方法と比較し効率良く優れた解を求められると言われる方法

In [24]:
# 問題設定 - 乳がんに関するデータセットを使用し,目標値が陰性か陽性かの二つの値である2値分類の問題設定
# 1,手動での調整
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

In [25]:
dataset = load_breast_cancer()
print(dataset)

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]]), 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

In [26]:
t = dataset.target #目標値
x = dataset.data #入力値

print(x.shape)
print(t.shape)

(569, 30)
(569,)


In [27]:
# 3tに分ける
# 1,学習用データセット(train)
# 2,検証用データセット(validation)
# 3,テスト用データセット(test)

# 割合はデータセットの量に依存するので決まりはない7:3.8:2が一般的
# 与えられたデータを"テスト用データセット:その他 = 20:80"に分割
# "その他"のデータを"検証用データセット:学習用データセット=30:70"に分割

from sklearn.model_selection import train_test_split

# 与えられたデータを"テスト用データセット:その他 = 20:80"に分割
x_train_val, x_test, t_train_val, t_test = train_test_split(x,t,train_size=0.2,random_state=1)

print(x_train_val.shape)
print(x_test.shape)
print(t_train_val.shape)
print(t_test.shape)

(113, 30)
(456, 30)
(113,)
(456,)


In [28]:
# "その他"のデータを"検証用データセット:学習用データセット=30:70"に分割
 
# その他のデータ(x_train_val,t_train_val)が混ざっているものを検証用データセットと学習用データセットに分割する必要がある
x_train, x_val, t_train, t_val = train_test_split(x_train_val,t_train_val,train_size=0.3,random_state=1)
# これでその他に入っていたxとtをtrainとvalに分けることができます

print(x_train.shape)
print(x_val.shape)
print(t_train.shape)
print(t_val.shape)

(33, 30)
(80, 30)
(33,)
(80,)


In [29]:
# 決定木を実装
from sklearn.tree import DecisionTreeClassifier

In [33]:
# ハイパーパラメータを入れる前
# dtree = DecisionTreeClassifier(random_state=0)

# ハイパーパラメータを入れた後
dtree = DecisionTreeClassifier(max_depth = 10,min_samples_split = 30, random_state=0)

In [34]:
dtree.fit(x_train,t_train)

DecisionTreeClassifier(max_depth=10, min_samples_split=30, random_state=0)

In [35]:
print(f"train score:{dtree.score(x_train,t_train)}")
print(f"val score:{dtree.score(x_val,t_val)}")
# 結果 - ハイパーパラメータの調整を行いモデルの学習を行う
# train score:1.0
# val score:0.7125

# dtree = DecisionTreeClassifier(random_state=0)にハイパーパラメータの設定を記述する

# ハイパーパラメータの調整後
# train score:0.9393939393939394
# val score:0.8375

train score:0.9393939393939394
val score:0.8375


In [ ]:
# 2,グリッドサーチ 
# 手動で適当に入れた値が常に最適なハイパーパラメータである可能性は低い
# そのため最適なハイパーパラメータを獲得するためにはある程度の探索、つまり試行錯誤を行う必要がある
# 効率的にハイパーパラメータを探索する方法はいくつかあり,その中の一つがグリッドサーチ 

# グリッドサーチ 
# 1.ハイパーパラメータを探索する範囲を決める,範囲の指定に決まりはない
# 2,その決められた範囲を活用して学習、検証を行う
# 3,その結果から予測精度が最も高いパラメータを採用する採用する

# メリット - 指定した範囲を網羅するため,ある程度漏れがなくハイパーパラメータの探索を行うことができる
# デメリット - 場合によっては,数十~数百パターンの組み合わせを計算するための学習に時間を要する

# グリッドサーチの実装 

In [47]:
from sklearn.model_selection import GridSearchCV

# GridSearchCVを使用するにはするには以下が必要
# 1,estimater - 学習に使用するモデル
# 2,param_grid - ハイパーパラメータの探索する範囲
# 3,CV - K-分割交差検証のKの値

In [51]:
# 1,estimater - 学習に使用するモデル
estimator = DecisionTreeClassifier(random_state=0)


# 2,param_grid - ハイパーパラメータの探索する範囲
# key - 調整するハイパーパラメータの名前
# value - リスト型の探索する範囲

param_grid = [{
    "max_depth": [3,20,50], #ここは今は適当
    "min_samples_split" : [3,20,30] #ここは今は適当
}]


# 3,CV - K-分割交差検証のKの値
cv=5

In [56]:
# 3,CVではK-分割交差検証が行われる。そのため学習用データセットと検証用データセットに分割する前のデータセットである、x_train_valとt_train__valを使用する
# return_train_score_falseという引数を設定することで学習に対する予測精度の検証が行われない

#モデルの宣言
tuned_model = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           cv=cv,
                           return_train_score=False)

In [58]:
#モデルの学習と検証
tuned_model.fit(x_train_val,t_train_val)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
             param_grid=[{'max_depth': [3, 20, 50],
                          'min_samples_split': [3, 20, 30]}])

In [ ]:
#学習結果
# cv_results_ - cross validationした結果が入っている
# .T - 転置すると実行結果が見やすくなる

pd.DataFrame(tuned_model.cv_results_).T

# param_grid = [{
#     "max_depth": [3,20,50], #ここは適当
#     "min_samples_split" : [3,20,30] #ここは適当
# }]

# 3*3の学習結果 - 合計9回のtry and errorが行われた
# mean_fit_time - 学習時間の平均
# std_fit_time - 学習時間の標準偏差
# mean_test_score(モデルの精度の確認) - 検証用データセットに対しての予測精度の平均

# この結果を参照して,先ほどより狭い範囲でハイパーパラメータの調整をする

In [60]:
# この結果を参照して,先ほどより狭い範囲でハイパーパラメータの調整をする
#　また新しい値を入れる(感覚を狭めた - これで精度を高めることができるかもしれない)

param_grid = [{
    "max_depth": [5,10,15], #ここは今は適当
    "min_samples_split" : [10,12,15] #ここは今は適当
}]

In [61]:
#モデルの宣言
tuned_model = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           cv=cv,
                           return_train_score=False)

In [62]:
#モデルの学習と検証
tuned_model.fit(x_train_val,t_train_val)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
             param_grid=[{'max_depth': [5, 10, 15],
                          'min_samples_split': [10, 12, 15]}])

In [63]:
#学習結果
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.00169892,0.00136046,0.00122075,0.00118365,0.00123591,0.00118504,0.00122781,0.00120044,0.00257978
std_fit_time,0.000263331,0.000285908,8.52351e-05,6.10278e-05,7.80499e-05,6.37181e-05,8.54472e-05,6.4429e-05,0.00281853
mean_score_time,0.000449228,0.000325108,0.000318718,0.000314379,0.000347948,0.000330782,0.000322771,0.000314236,0.000320578
std_score_time,8.45087e-05,1.84707e-05,7.0588e-06,8.06207e-06,6.69827e-05,3.28805e-05,2.39605e-05,1.5325e-05,1.27295e-05
param_max_depth,5,5,5,10,10,10,15,15,15
param_min_samples_split,10,12,15,10,12,15,10,12,15
params,"{'max_depth': 5, 'min_samples_split': 10}","{'max_depth': 5, 'min_samples_split': 12}","{'max_depth': 5, 'min_samples_split': 15}","{'max_depth': 10, 'min_samples_split': 10}","{'max_depth': 10, 'min_samples_split': 12}","{'max_depth': 10, 'min_samples_split': 15}","{'max_depth': 15, 'min_samples_split': 10}","{'max_depth': 15, 'min_samples_split': 12}","{'max_depth': 15, 'min_samples_split': 15}"
split0_test_score,0.782609,0.782609,0.782609,0.782609,0.782609,0.782609,0.782609,0.782609,0.782609
split1_test_score,1,1,1,1,1,1,1,1,1
split2_test_score,0.956522,0.956522,0.956522,0.956522,0.956522,0.956522,0.956522,0.956522,0.956522


In [64]:
# モデルの予測精度を確認する
# 最も予測精度の高かったハイパーパラメータを確認する方法
tuned_model.best_params_

{'max_depth': 5, 'min_samples_split': 10}

In [66]:
# 学習したモデル自体を引き継ぎたい場合
best_model = tuned_model.best_estimator_

In [67]:
print(best_model.score(x_train_val,t_train_val))
print(best_model.score(x_test,t_test))

# 結果 - 手動
# train score:0.9393939393939394
# val score:0.8375

# 結果 - グリッドサーチ
# 1.0
# 0.881578947368421

1.0
0.881578947368421
